In [1]:
import os

import re
from operator import itemgetter

from utils import split_text_at_punctuation

from langchain.document_loaders.text import TextLoader

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.chat_models.openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.pydantic_v1 import BaseModel, Field

from langchain.vectorstores import qdrant

from config import GPT_API

os.environ["OPENAI_API_KEY"] = GPT_API


In [2]:
SET_TEMPERATURE = 0.3
MODEL = "gpt-4o-2024-05-13"#"gpt-3.5-turbo"

LLM_MODEL = ChatOpenAI(
    model=MODEL,
    name="Agent for job applications",
    temperature=SET_TEMPERATURE,
    n=1)

/home/mangabat/miniconda3/envs/bumstuff/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
# loader = TextLoader("ai_engineer_software_engineer.txt")

loader = DirectoryLoader(path="jobtemplates/")

load_applications = loader.load()

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    encode_kwargs = {'normalize_embeddings':False}
)

vectorstore = qdrant.Qdrant.from_documents(
    documents=load_applications,
    embedding=embedding_model,
    location=":memory:"
)


/home/mangabat/miniconda3/envs/bumstuff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
skills = [
    "Business analytics",
    "Business maturity",
    "Strategy",
    "Non-technical and technical communication",
    "Algorithms & datastrucures",
    "Software Engineering",
    "detail oriented",
    "Creative thinker",
    "Problem solving",
    "Critical thinking",
    "Team player",
    "Time management",
    "Adaptability",
    "Conflict resolution",
    "Collaborative",
    "Dilligent"
]

IT_Management = [
    "ITIL",
    "SAFe",
    "PRINCE2",
    "CMMI",
    "SCRUM",
    "Agile development",
    "UML(frequency, class or C4)",
    "Stakeholder classification"
]

Programming_languages = [
    "Python intermediate level",
    "SQL working understanding",
    "R working understanding",
    "JavaScript working understanding"
]

technical_skills = [
    "git"
    "Statistical modelling",
    "Fundamental Azure knowledge",
    "PostGres",
    "Neo4J",
    "Qdrant",
    "ANNOY",
    "Docker",
    "scraping",
    "crawling",
    "MT5",
    "Bert",
    "FinBert",
    "T5",
    "Scrapy",
    "Numpy",
    "Polars",
    "Pandas",
    "FastAPI",
    "VUE3",
    "TensorFlow2",
    "Hyggingface",
    "Pytorch",
    "SonarCube",
    "Seaborn(/matplotlib/Plotly)",
    "PyTest",
    "SKlearn"
    "Unsupervised learning: dimensionality reduction, explorative factor analysis, K-mean..",
    "Supervised learning: Random Forests, multiple logistic regression, SVP, NNs, Classification"
]

skills_dict = {
    'soft skills': skills,
    'IT Management': IT_Management,
    'Programming languages': Programming_languages,
    'Technical skills': technical_skills
}

do_not_use_words = [
    "abreast",
    "ardent",
    "cruisal",
    "deeply",
    "eagerly",
    "endeavors",
    "enhance",
    "enhanced",
    "enhancing",
    "extensive",
    "extensively", 
    "expert",
    "expertise",
    "forefront",
    "fostering",
    "fueled",
    "fulfilling",
    "honed",
    "intricacies",
    "intricate",
    "meticulous ",
    "perfect",
    "perfectly",
    "prowess",
    "profoundly",
    "realm",
    "seamlessly",
    "specialist",
    "stems",
    "thrilled",
    "versed"
]


In [5]:
query_for_search = """
VI SØGER EN DATA SCIENTIST, DER VIL GÅ FORAN SAMMEN MED OS 

I Aarhus Kommune er vi ambitiøse og går forrest, når vi skaber fundamenterne for fremtidens velfærd. Også når det gælder ibrugtagning af teknologi, der kan hjælpe os med at opnå vores mål om at være en god by for alle. Derfor søger vi en ambitiøs – men gerne nyuddannet – data scientist, der har lyst til at træde nye stier sammen med os.

Vi søger en data scientist, der kan lide at gå på opdagelse i data og se muligheder sammen med kolleger – enten i teamet eller på tværs af kommunen. Vi søger en, der har erfaring med at udvikle, træne og evaluere ML-modeller. En, der er nysgerrig på nye metoder og værktøjer. Særlig har vi lige nu fokus på sprogmodeller (hvem har ikke det?). Vi er indgået i et samarbejde med en række aktører om at skabe en ny, dansk sprogmodel i Dansk Sprogmodel Konsortium, og det er blandt andet derfor, vi særligt har brug for dig lige nu! Der er dog også mange andre områder, vi gerne vil have, du kaster dig over – det kunne være klassificering af henvendelser i vores support, understøttelse af vores juridiske afdeling, arbejdet med at automatisere referater mv. Mulighederne er (næsten) uendelige!

Vi tilbyder dig en fantastisk arbejdsplads med gode arbejdsforhold, godt kollegialt sammenhold, højt til loftet og med stor mulighed for at præge dit nye arbejde som data scientist i Borgmesterens Afdeling.

I Borgmesterens Afdeling er vi ambitiøse og forsøger at rykke hurtigt på aktuelle og værdiskabende dagsordener. Derfor vil du opleve en omskiftelig hverdag, hvor du kan bruge din faglighed på mange forskellige måder.

Masser af gode, dygtige kolleger – både i teamet og på tværs af kommunen

I sektionen Digitale Fundamenter, hvor du vil blive placeret, sidder dygtige og erfarne kolleger, der arbejder med data og arkitektur, som understøtter hele Aarhus Kommunes digitale processer. Sektionen indeholder blandt andet et datateam, der leverer data til mange forskellige formål fx dataintegrationer til tværgående fagsystemer, data til analyse, rapportering og ledelsesinformationsløsninger. Vores data og processer er i høj grad forankret omkring Microsoft SQL Server og af den årsag bruger vi også en stribe almene Microsoft-værktøjer. I sektionen har man allerede længe arbejdet med at forberede arbejdet med machine learning og understøttet andre afdelinger, men det er først med dig, at vi for alvor tager hul på udviklingsopgaven lokalt. Vi har dog allerede mange kolleger, der sidder i starthullerne for at galoppere afsted sammen med dig!

Du får en central rolle i at opbygge området for kunstig intelligens i Borgmesterens Afdeling – bl.a. vil du, i samarbejde med dine kolleger, etablere relevante data- og udviklingsplatforme og opbygge vores kompetencer inden for data science og klargøring af data til understøttelse af ibrugtagningen af AI-teknologier.

Derudover vil du, med din data science-erfaring, faglighed og indsigt, i samarbejde med vores konsulenter, være med til at vurdere og udpege de indsatsområder, hvor vi bedst anvender kunstig intelligens. Udover at anvende sprogmodeller til at effektivisere og kvalitetssikre den administrative opgave, ser vi ind i områder som økonomi, jura, it-support mv., hvor vi har ambitioner om at benytte machine learning. Mulighederne i Borgmesterens Afdeling og resten af kommunen er mangfoldige og derfor er det vigtigt, at du hjælper med at udvælge få attraktive indsatsområder, som vi kommer i dybden med og ser resultater af. Dette gælder i særlig grad i forhold til vores arbejde med at levere usecases og klargjorte data til brug i og sammen med Dansk Sprogmodel Konsortium.

Din vigtigste opgave er at arbejde med vores data og udvikle, træne og evaluere modeller. Dette i tæt samarbejde med bl.a. dine kolleger i sektionen og vores driftsområder, hvor sidstnævnte har dyb faglighed og indsigt i de faglige processer.

Idet stillingen er ny, så har du rig mulighed for at udforme og påvirke indholdet. Bl.a. valg af platforme, programmeringssprog, frameworks, indsatsområder mv. Aarhus Kommune er ved at afslutte en helhedsorienteret AI-analyse, der vil præsentere en række principper, som vi i kommunen skal arbejde efter – herunder vil der være en opgave i at implementere de beslutninger og retning, som er givet strategisk sammen med gode ledere og kolleger.

Dine nærmeste kolleger har du i sektionen, hvor du ansættes. Du bliver en del af datateamet, og vil derfor hurtigt få et dybt kendskab til vores nuværende dataområder, arbejdsopgaver og metoder. Derudover har du tætte samarbejdsrelationer med dine kolleger fra sektionerne Strategi og Styring og Servicedesign.

Og så har vi som nævnt mange samarbejdsfora på tværs af kommunen, hvor der sidder dygtige kolleger (blandt andet en taskforce, der arbejder med koordinering af AI-indsatser på tværs af kommunen samt et udviklernetværk for andre data scientists mv.), der arbejder med store, ambitiøse projekter. Både inden for Aarhus Kommunes mure og på tværs af andre kommuner. Med virksomheder, offentlige myndigheder og universiteter. Med andre nordiske byer og i større EU-projekter.

Dine kompetencer

De Kompetencer, Vi Gerne Ser Du Har, Er

en relevant uddannelse, fx indenfor computer science, matematik, statistik eller lignende. 
erfaring med data science/machine learning – evt. fra tidligere job, herunder tilrettelæggelse af datasæt, der egner sig til træning af både simple og mere komplekse modeller 
erfaring med at opstille hypoteser på såvel struktureret som ustruktureret data, og du kan udtrykke dine analytiske metoder i et eller flere passende programmeringssprog 
erfaring med at udvikle, træne og evaluere modeller, og forstår vigtigheden af at gøre det på en systematisk og reproducerbar måde 
at du måske særligt interesserer dig for/har erfaring med NLP, snarere end andre typer ML/AI, da det særligt er tekstdata, vi ser et potentiale i at arbejde med 
at du er struktureret, analytisk og har en god forestillingsevne, du bruger til at finde løsninger på problemstillinger ”i den virkelige verden” 
at du er kreativ, iderig og elsker at udvikle nye løsninger i samarbejde med andre fagligheder 
at du er god til at kommunikere komplekse problemstillinger til bl.a. ledelse og samarbejdspartnere 
at du har forståelse for etiske og juridiske problemstillinger ifm. data science og kunstig intelligens som bredt begreb 
at du evt. har erfaring med Microsoft-infrastruktur 
at du evt. har erfaring med at ”deploye” virksomme modeller i produktionsmiljøer - men da du måske er nyuddannet, er dette ikke noget, vi forventer. 

Velkommen til afdelingen Fælles It og Digitalisering

Vi Tilbyder Dig

et dynamisk arbejdsmiljø med en bred opgaveportefølje 
en god, engageret og sjov kollegaflok og ledere i en afdeling med uformel omgangstone og et stort fokus på socialt samvær 
høj grad af medindflydelse på din opgavesammensætning 
udfordrende opgaver, der sørger for, at du udvikler dine kompetencer 
mulighed for faglig sparring ved både kolleger og leder. 
fleksible arbejdstider og god balance mellem arbejds- og privatliv. 

Derfor Skal Du Vælge Os

Med et job i Borgmesterens Afdeling er du tæt på beslutningerne i landets næststørste by.

Du kommer til at arbejde i de centrale it-afdelinger i Aarhus Kommune, som har opgaver, der går tværs af organisationen. Vi skaber fremtidssikrede it-løsninger og stabil drift for både medarbejdere og borgere. 

Når du arbejder for os, vil du møde: 

Faglige ambitioner 
Komplekse opgaver 
Stor tillid og fleksibilitet i hverdagen 

Se mere om hvem vi er, og hvad vi laver på digital.aarhus.dk. 
"""
semilarity_document_template = vectorstore.similarity_search_with_score(
    query = query_for_search,
    k = 1,
    score_threshold=0.1)

print("semilarity_document_template", semilarity_document_template)

semilarity_document_template [(Document(page_content="Data Scientist\n\nI reached out due to my interest in AI, Software Development/Engineering, and Knowledge management.\n\nMotivation I am driven to progress in the field of AI, particularly focusing on Large Language Models, knowledge extraction, and Natural Language Processing. My goal is to ensure that every AI implementation has a solid foundation of reliable data. I am dedicated to setting up thorough data agreements to back these efforts and am excited about the prospect of working alongside your team. My passion for AI is motivated by genuine curiosity and the satisfaction derived from tackling complex problems. Whether working independently or as part of a team, I am committed to addressing every necessary element to deliver a well-rounded AI solution, including aspects of security and compliance with GDPR and AI-ACT. I am enthusiastic about engaging in an environment that encourages detailed and strategic discussions on AI ap

In [6]:
VACANCY_ANALYSIS_PROMT = ChatPromptTemplate.from_messages(
    [
        ('system',"""
                You are an assisatant to a human resource manager//
                You are to assist in the analysis of a job vacancy//
                Identify vocal points of interest that the company is looking for//
                Identify the company name//
                Identify the job title//
                Identify the skills and technical experience required for the job vacancy provided here//
                identify the skills and requirements for the job vacancy//
        """),

        ('human',"""
                Given the job vacancy, you are to analyse the following in detail: {SomeVacantPosition}//
                Use these skills {my_skills} to conduct an analysis between job requirements and find matching skills//
                Output should contain a list of matching skills required for the job vacancy//
                {format_instructions_1}
        """),
    ]
)

## Output data structure
class OutputStuctureV1(BaseModel):
    company_name: str = Field(description="identified company name")
    job_title: str = Field(description="identified job title")
    analysis_output: str = Field(description="analysis of the job vacancy")
    employees_skills_requirement: dict = Field(description="identified skills and technical experience required for the job vacancy")
    matching_skills: dict = Field(description="matching skills in the job vacancy")

parser_1 = PydanticOutputParser(pydantic_object=OutputStuctureV1)

format_messages = VACANCY_ANALYSIS_PROMT.format(
    SomeVacantPosition = query_for_search,
    my_skills = skills_dict,
    format_instructions_1 = parser_1.get_format_instructions())

chain = LLM_MODEL | parser_1 

analysis_chain = chain.invoke(format_messages)


In [7]:
for i in analysis_chain:
    print(i)


print(analysis_chain.analysis_output)

('company_name', 'Aarhus Kommune')
('job_title', 'Data Scientist')
('analysis_output', 'The job vacancy for a Data Scientist at Aarhus Kommune focuses on developing, training, and evaluating machine learning models, with a particular emphasis on natural language processing (NLP). The role involves working with data and developing AI technologies to support various departments within the municipality. The candidate should have experience in data science, machine learning, and be able to work collaboratively with other teams. The job offers a dynamic work environment with opportunities for professional growth and flexible working conditions.')
('employees_skills_requirement', {'education': 'Relevant education in computer science, mathematics, statistics, or similar fields.', 'experience': ['Experience with data science/machine learning.', 'Experience in developing, training, and evaluating models.', 'Experience with NLP and text data.', 'Experience with Microsoft infrastructure (optional

In [8]:
# Extracting identified information from analysis_chain
identified_company_name = analysis_chain.company_name
identified_job_title = analysis_chain.job_title
identified_skill_requirements = analysis_chain.employees_skills_requirement
identified_matching_skills = analysis_chain.matching_skills
identified_analysis_output = analysis_chain.analysis_output

# Create the output dictionary according to the schema
output = {
    "company_name": identified_company_name,
    "job_title": identified_job_title,
    "analysis_output": identified_analysis_output,
    "employees_skills_requirement": identified_skill_requirements
}

# Getting keys from the dictionary
get_employee_requirements_keys = identified_skill_requirements.keys()

# Create an itemgetter object with these keys
get_employee_requirements_lists = itemgetter(*get_employee_requirements_keys)

# Applying itemgetter to the dictionary to get the lists
employee_requirements = get_employee_requirements_lists(identified_skill_requirements)

# Zipping keys with their corresponding lists
lists_with_employee_requirements = zip(get_employee_requirements_keys, employee_requirements)

# If you need to process lists_with_employee_requirements further, you can do so here
get_matching_skills_keys = identified_matching_skills.keys()
get_matching_skills_lists = itemgetter(*get_matching_skills_keys)
matching_skills = get_matching_skills_lists(identified_matching_skills)
lists_with_matching_skills = zip(get_matching_skills_keys, matching_skills)
# Example usage:
# print the output dictionary
print(list(lists_with_employee_requirements))
print(list(lists_with_matching_skills))





[('education', 'Relevant education in computer science, mathematics, statistics, or similar fields.'), ('experience', ['Experience with data science/machine learning.', 'Experience in developing, training, and evaluating models.', 'Experience with NLP and text data.', 'Experience with Microsoft infrastructure (optional).', 'Experience with deploying models in production environments (optional).']), ('skills', ['Ability to work with structured and unstructured data.', 'Analytical and systematic approach to problem-solving.', 'Creativity and ability to develop new solutions.', 'Good communication skills.', 'Understanding of ethical and legal issues in data science and AI.']), ('technical_skills', ['Programming languages (e.g., Python, SQL).', 'Knowledge of Microsoft SQL Server and related tools.', 'Experience with machine learning frameworks and tools.'])]
[('soft_skills', ['Business analytics', 'Non-technical and technical communication', 'detail oriented', 'Creative thinker', 'Problem 

In [9]:
############# TO FIX

## add a check for the length of the output


TEXT_GENERATION_PROMPT = ChatPromptTemplate.from_messages(
    [
        ('system',"""
         You are to assist in setting up a job application template//
         The total amount of characters that can be used is 4000, include white spaces//
         200-300 characters for the introduction section//
         800-1000 characters for the motivation section//
         500-800 characters for the skills section//
         560 characters for the masters section//
         390 characters for the bachelors section//
         300 characters for the continued learning section//
         200 characters for the thank you note//
         Grammatical correctness is essential//
         Use casual business language//
         Ensure, the English language is equal to EITLS c1 score//
         The template job application must be in English//
         
         This template is the jobtemplate: {semilarity_jobtemplate}// 
         
        """),

        ('human',"""
         
         I have the following knowledge and skills which can be found in the following dictionary {skills}//
         
         write two lines to generate a short introduction with interest in IT and AI with inspiration from the {analysis_output}//
         
         write motivation with matching pairs {skill_match} and {employee_requirements} and how these can be utilized for the company's benefi//
         
         write a section about skills somme of the skills and how they can be utilized for the company's benefit//

         keep educational background for later access and save the section about masters degree into latex_edu_master and the section about bachelors into latex_edu_bachelor//
         keep continued learning section and provide short context that I am willing to learn what is necessary for the company and specific role//
        
         write a short and consice thank you note to setup a cofee//

         I DO NOT have prior experience in a professional environment in programming, ONLY academia//
         I DO have prior experience in project management//
         {format_instructions_2}
        """),
    ]
)

## Output data structure
class OutputStuctureV2(BaseModel):
    latex_company_name: str = Field(description="Company name")
    latex_job_title: str = Field(description="Job title")
    latex_introduction: str = Field(description="Introduction")
    latex_motivation: str = Field(description="Motivation")
    latex_skills: str = Field(description="Skills")
    latex_edu_masters: str = Field(description="Masters")
    latex_edu_bachelor: str = Field(description="Bachelor")
    latex_continued_learning: str = Field(description="Continued learning")
    latex_thank_you: str = Field(description="Thank you for your time")


parser_2 = PydanticOutputParser(pydantic_object=OutputStuctureV2)

format_messages_2 = TEXT_GENERATION_PROMPT.format(
    analysis_output = identified_analysis_output,
    employee_requirements = lists_with_employee_requirements,
    skill_match = lists_with_matching_skills,
    skills = skills_dict,
    semilarity_jobtemplate = semilarity_document_template,
    format_instructions_2 = parser_2.get_format_instructions())

print(format_messages_2)
chain_2 = LLM_MODEL | parser_2

analysis_chain_2 = chain_2.invoke(format_messages_2)

System: 
         You are to assist in setting up a job application template//
         The total amount of characters that can be used is 4000, include white spaces//
         200-300 characters for the introduction section//
         800-1000 characters for the motivation section//
         500-800 characters for the skills section//
         560 characters for the masters section//
         390 characters for the bachelors section//
         300 characters for the continued learning section//
         200 characters for the thank you note//
         Grammatical correctness is essential//
         Use casual business language//
         Ensure, the English language is equal to EITLS c1 score//
         The template job application must be in English//
         
         This template is the jobtemplate: [(Document(page_content="Data Scientist\n\nI reached out due to my interest in AI, Software Development/Engineering, and Knowledge management.\n\nMotivation I am driven to progress in

In [10]:
validate_company_name = analysis_chain_2.latex_company_name
validate_jobtitle = analysis_chain_2.latex_job_title
validate_introduction = analysis_chain_2.latex_introduction
validate_motivation = analysis_chain_2.latex_motivation
validate_skills = analysis_chain_2.latex_skills
validate_masters = analysis_chain_2.latex_edu_masters
validate_bachelors = analysis_chain_2.latex_edu_bachelor
validate_continued_learning = analysis_chain_2.latex_continued_learning
validate_thank_you = analysis_chain_2.latex_thank_you



In [11]:
############# TO FIX
## add few-shot classification to ensure the output is correct
## add a check for the forbidden words
## add a check for the length of the output


REMOVE_WORDS_PROMPT = ChatPromptTemplate.from_messages(
    [
        ('system',"""
         keep the original text here {analysis_chain_2} and substitube words if they appear on this list {forbidden_words} with something similar //
        {format_instructions_3}
         

        """),
    ]
)

parser_3 = PydanticOutputParser(pydantic_object=OutputStuctureV2)

format_messages_3 = REMOVE_WORDS_PROMPT.format(
    analysis_chain_2 = analysis_chain_2,
    forbidden_words = do_not_use_words,
    format_instructions_3 = parser_3.get_format_instructions())

print(format_messages_3)
chain_3 = LLM_MODEL | parser_3

analysis_chain_3 = chain_3.invoke(format_messages_3)


System: 
         keep the original text here latex_company_name='Aarhus Kommune' latex_job_title='Data Scientist' latex_introduction="I am excited to apply for the Data Scientist position at Aarhus Kommune. My passion for IT and AI, particularly in developing and evaluating machine learning models, aligns perfectly with the role's focus on natural language processing (NLP)." latex_motivation='My motivation to join Aarhus Kommune stems from my deep interest in AI and machine learning, particularly in NLP. My academic background has equipped me with the skills to develop, train, and evaluate machine learning models. I am particularly driven by the opportunity to work on projects that support various departments within the municipality, ensuring that AI technologies are effectively integrated to enhance public services. My experience in project management has honed my ability to work collaboratively with diverse teams, ensuring that project goals are met efficiently. I am eager to bring 

In [12]:
for i in analysis_chain_3:
    print(i)

('latex_company_name', 'Aarhus Kommune')
('latex_job_title', 'Data Scientist')
('latex_introduction', "I am excited to apply for the Data Scientist position at Aarhus Kommune. My passion for IT and AI, particularly in developing and evaluating machine learning models, aligns seamlessly with the role's focus on natural language processing (NLP).")
('latex_motivation', 'My motivation to join Aarhus Kommune arises from my deep interest in AI and machine learning, particularly in NLP. My academic background has equipped me with the skills to develop, train, and evaluate machine learning models. I am particularly driven by the opportunity to work on projects that support various departments within the municipality, ensuring that AI technologies are effectively integrated to improve public services. My experience in project management has refined my ability to work collaboratively with diverse teams, ensuring that project goals are met efficiently. I am eager to bring my skills in data scien

In [13]:
final_company_name = analysis_chain_3.latex_company_name
final_jobtitle = analysis_chain_3.latex_job_title
final_introduction = analysis_chain_3.latex_introduction
final_motivation = analysis_chain_3.latex_motivation
final_skills = analysis_chain_3.latex_skills
final_masters = analysis_chain_3.latex_edu_masters
final_bachelors = analysis_chain_3.latex_edu_bachelor
final_continued_learning = analysis_chain_3.latex_continued_learning
final_thank_you = analysis_chain_3.latex_thank_you


def validate_words(do_not_use_words, *args):
    false_count = 0
    true_count = 0

    forbidden_words_used = []

    for arg in args:
        words = arg.split()
        for word in words:
            if word in do_not_use_words:
                false_count += 1
                forbidden_words_used.append(word)
            else:
                true_count += 1
    
    # Return a tuple with the counts and the list of forbidden words used
    return true_count, false_count, forbidden_words_used

true_count, false_count, forbidden_words_used = validate_words(
    do_not_use_words,
    final_company_name, final_jobtitle, final_introduction, final_motivation, 
    final_skills, final_masters, final_bachelors,  final_continued_learning, 
    final_thank_you
)

# Print the results
print(f"Number of allowed words: {true_count}")
print(f"Number of forbidden words used: {false_count}")
print(f"Forbidden words used: {forbidden_words_used}")

Number of allowed words: 469
Number of forbidden words used: 2
Forbidden words used: ['seamlessly', 'enhancing']


In [14]:


#some regex to remove characters that intervene with latex commands

def check_latex_safety(*args):
    # Dictionary to map LaTeX special characters to their safe equivalents
    replacements = {
        '\\': ' ',          # backslash to space
        '{': ' ',           # curly brace to space
        '}': ' ',           # curly brace to space
        '#': ' ',           # hash to space
        '%': ' ',           # percent to space
        '&': 'and',         # ampersand to 'and'
        '_': ' ',           # underscore to space
        '^': ' ',           # caret to space
        '~': ' ',           # tilde to space
        '$': 'dollars',     # dollar to space
        '/': ' ',           # slash to space
        '*': ' ',           # asterisk to space
        '-': ' '            # hyphen to space
    }
    
    # Regex pattern to match any LaTeX special character
    pattern = r'[\\{}#%&_^\~$\/\*\-]'
    
    true_count = 0
    false_count = 0

    # Function to replace matched characters
    def replace_match(match):
        return replacements[match.group(0)]
    
    # Process each input text
    results = []
    for text in args:
        if re.search(pattern, text):
            false_count += 1
            safe_text = re.sub(pattern, replace_match, text)
        else:
            true_count += 1
            safe_text = text
        results.append(safe_text)

    print(f"Number of safe texts: {true_count}")
    print(f"Number of modified texts: {false_count}")

    return results


# Example usage:

(final_company_name, 
 final_jobtitle, 
 final_introduction, 
 final_motivation, 
 final_skills, 
 final_masters, 
 final_bachelors, 
 final_continued_learning, 
 final_thank_you) = check_latex_safety(
                                        final_company_name, 
                                        final_jobtitle, 
                                        final_introduction, 
                                        final_motivation, 
                                        final_skills, 
                                        final_masters, 
                                        final_bachelors, 
                                        final_continued_learning, 
                                        final_thank_you)





# Directory where the variables.tex file will be saved
directory = 'latex'

# Create the directory if it does not exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Write these variables to a .tex file in the specified directory
variables_file_path = os.path.join(directory, 'variables.tex')
with open(variables_file_path, 'w') as text_for_latex:
    text_for_latex.write(f"\\newcommand{{\\finalCompanyName}}{{{final_company_name}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalJobtitle}}{{{final_jobtitle}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalIntroduction}}{{{final_introduction}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalMotivation}}{{{final_motivation}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalSkills}}{{{final_skills}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalEducationMaster}}{{{final_masters}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalEducationBachelor}}{{{final_bachelors}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalContinuedLearning}}{{{final_continued_learning}}}\n")
    text_for_latex.write(f"\\newcommand{{\\finalThankYou}}{{{final_thank_you}}}\n")



Number of safe texts: 4
Number of modified texts: 5
